In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])    
])

transform_test = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])    

])

train_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/dataset/train', transform=transform_train)
test_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/dataset/test', transform=transform_test)

partition = {'train': train_dataset, 'test':test_dataset}

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import numpy as np
import argparse
from copy import deepcopy
import time


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_dim, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_channels, out_dim, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_dim) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(out_dim, out_dim, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_dim) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우 -> 굳이 이 부분을 sequential로 만들어야 하는가?
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_dim, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_dim)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes, in_channels, stride=1):
        super(ResNet, self).__init__()
        self.in_channel = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_dim, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # 첫 번째 stride 외에 나머지 stride는 1로 고정 
        layers = []
        for stride in strides:
            layers.append(block(self.in_channel, out_dim, stride))
            self.in_channel = out_dim # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = self.linear(out)
        return out

In [ ]:
def train(net, partition, optimizer, criterion, args):
    train_loader = torch.utils.data.DataLoader(partition['train'], batch_size=args.train_batch_size, shuffle=True, num_workers=4)

    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        inputs, targets = data
        inputs = inputs.cuda()
        targets = targets.cuda()
        outputs = net(inputs)

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1) # 1. row 값 중에 최댓값 (_ 처리해서 무시)
                                             # 2. 최댓값의 column index  (predicted로 받아옴)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [ ]:
def validate(net, partition, criterion, args):
    val_loader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=args.test_batch_size, 
                                            shuffle=False, num_workers=4)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in val_loader:
            inputs, targets = data
            inputs = inputs.cuda()
            targets = targets.cuda()
            outputs = net(inputs)

            loss = criterion(outputs, targets)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [ ]:
def test(net, partition, args):
    test_loader = torch.utils.data.DataLoader(partition['test'], batch_size=100, shuffle=False, num_workers=4)

    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            inputs, targets = data
            inputs = inputs.cuda()
            targets = targets.cuda()
            outputs = net(inputs)

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0) # batch_size

        test_acc = 100 * correct / total

    return test_acc

In [ ]:
def experiment(partition, args):

    net = ResNet(block = args.block,
                 num_blocks = args.num_blocks,
                 num_classes = args.num_classes,
                 in_channels = args.in_channels,
                 stride = args.stride
                 )
    
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        # val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        # val_losses.append(val_loss)
        train_accs.append(train_acc)
        # val_accs.append(val_acc)
   
        #print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        print('Epoch {}, Acc(train): {:2.2f}, Loss(train) {:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, train_loss, te-ts))
        
    test_acc = test(net, partition, args)    
    
    result = {}
    result['train_losses'] = train_losses
    #result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    #result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    #result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

In [ ]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr_model_code"

# ====== Model ====== #
args.in_channels = 3
args.block = BasicBlock
args.num_blocks = [2, 2, 2, 2]
args.num_classes = 2
args.stride = 1

# ====== Regularization ======= #
args.l2 = 0.00001

# ====== Optimizer & Training ====== #
# args.optim = 'SGD' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 64
args.test_batch_size = 64

# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'optim'
list_var1 = [0.0001, 0.00001]
list_var2 = ['RMSprop', 'Adam']


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(partition, deepcopy(args))
        #save_exp_result(setting, result)